<a href="https://colab.research.google.com/github/oektomo/KelasRLG2/blob/master/DQL_Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install score

In [17]:
import random
import gym
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam

# from scores.score_logger import ScoreLogger

ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

In [3]:
class DQNSolver:
  def __init__(self, observation_space, action_space):
    self.exploration_rate = EXPLORATION_MAX
    
    self.action_space = action_space
    self.memory = deque(maxlen=MEMORY_SIZE)

    self.model = Sequential()
    self.model.add(Dense(24, input_shape=(observation_space, ), activation="relu"))
    self.model.add(Dense(24, activation="relu"))
    self.model.add(Dense(self.action_space, activation="linear"))
    # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    self.model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))
  
  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  # choose if we want to explore or use the neural net
  def act(self, state):
    if np.random.rand() < self.exploration_rate:
      return random.randrange(self.action_space)
    q_values = self.model.predict(state)
    return np.argmax(q_values[0])

  # Replay Memory in Experience Replay
  def experience_replay(self):
    if len(self.memory) < BATCH_SIZE:
      return
    batch = random.sample(self.memory, BATCH_SIZE)
    for state, action, reward, state_next, terminal in batch:
      q_update = reward
      if not terminal:
        q_update = reward + GAMMA * np.amax(self.model.predict(state_next)[0])
      q_values = self.model.predict(state)
      q_values[0][action] = q_update
      self.model.fit(state, q_values, verbose=0)
    self.exploration_rate *= EXPLORATION_DECAY
    self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [36]:
def cartpole(max_run = 3):
    env = gym.make(ENV_NAME)
    #score_logger = ScoreLogger(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    score = []
    #max_run = 3
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        cpos = []
        cvel = []
        pangl = []
        cpos.append(state[0,0])
        cvel.append(state[0,1])
        pangl.append(state[0,2])
        while True:
            step += 1
            #env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            cpos.append(state_next[0])
            cvel.append(state_next[1])
            pangl.append(math.degrees(state_next[2]))
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                print ("Episode: "+str(run)+
                       ", exploration: "+str(round(dqn_solver.exploration_rate, 3))+
                       ", score: "+str(step)+
                       ", pos: "+str(round(state_next[0,0], 3))+
                       ", angle: "+str(round(math.degrees(state_next[0,2]), 2))+
                       " deg")
                score.append(step)
                #plt.plot(pangl)
                #plt.plot(cvel)
                #score_logger.add_score(step, run)
                break
            dqn_solver.experience_replay()
        if run >= max_run:
            break
    plt.plot(score)

In [35]:
cartpole(100)

Episode: 1, exploration: 0.99, score: 22, pos: -0.079, angle: 12.34
Episode: 2, exploration: 0.928, score: 14, pos: 0.175, angle: -13.48
Episode: 3, exploration: 0.744, score: 45, pos: 0.108, angle: -12.44
Episode: 4, exploration: 0.64, score: 31, pos: -0.241, angle: 12.06
Episode: 5, exploration: 0.597, score: 15, pos: -0.203, angle: 12.24
Episode: 6, exploration: 0.559, score: 14, pos: -0.147, angle: 12.72
Episode: 7, exploration: 0.498, score: 24, pos: -0.148, angle: 13.36
Episode: 8, exploration: 0.479, score: 9, pos: -0.099, angle: 13.98
Episode: 9, exploration: 0.446, score: 15, pos: -0.221, angle: 14.48
Episode: 10, exploration: 0.42, score: 13, pos: -0.15, angle: 12.82
Episode: 11, exploration: 0.402, score: 10, pos: 0.152, angle: -12.08
Episode: 12, exploration: 0.384, score: 10, pos: 0.121, angle: -13.17
Episode: 13, exploration: 0.36, score: 14, pos: 0.212, angle: -12.8
Episode: 14, exploration: 0.346, score: 9, pos: 0.143, angle: -12.68
Episode: 15, exploration: 0.321, scor

KeyboardInterrupt: ignored

In [18]:
env.observation_space.shape[0]

4

In [13]:
env.action_space

Discrete(2)


    Observation: 
        Type: Box(4)

    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:
    | Num | Observation           | Min                  | Max                |
    |-----|-----------------------|----------------------|--------------------|
    | 0   | Cart Position         | -4.8                 | 4.8                |
    | 1   | Cart Velocity         | -Inf                 | Inf                |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°)  | ~ 0.418 rad (24°)  |
    | 3   | Pole Angular Velocity | -Inf                 | Inf                |
    **Note:** While the ranges above denote the possible values for observation space of each element, 
    it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:
    -  The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates if the cart leaves the `(-2.4, 2.4)` range.
    -  The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)


    Action:
        Type: Discrete(2)
        Num	Action
        0	Push cart to the left
        1	Push cart to the right